In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests as rq
import lxml 
import time
import pandas as pd
import numpy as np
import random

# Step 1: Get the URL of the webpage of every district in Tokyo.
Including the 23 special wards(特別区), cities(市), towns(町) , and villages(村).




Set up the ChromeDriver.

In [10]:
browser = webdriver.Chrome("C:/Users/Nolan/Desktop/.vscode/chromedriver")
browser.get('https://suumo.jp/chintai/tokyo/city/')
browser.maximize_window()


Get URLs

In [11]:
boxes = browser.find_elements_by_name('sc') #The list of all checkboxes
District_URLs = pd.DataFrame(columns=['ID', 'District', 'URL']) #Create a dataframe to store all the URLs
for i in range(0,len(boxes)):
    browser.get('https://suumo.jp/chintai/tokyo/city/') 
    boxes = browser.find_elements_by_name('sc') # Find all the checkboxes
    label = browser.find_elements_by_tag_name('label') #The name tag of each checkbox

    boxes[i].click() #Click the checkbox
    idd = boxes[i].get_attribute('id') #Record the ID of the checkboxes
    labelid = label[i].get_attribute('for') #Record the ID of the label

    if (labelid == idd): 
        district = label[i].text
        district = district.split('(')[0] #Make sure that each checkbox has a corresponding name tag 
    else:
        pass

    Search_button = browser.find_element_by_xpath('//*[@id="js-searchpanel"]/div/div/a') #The search button
    Search_button.click()
    url = browser.current_url #Get the url of current page
    District_URLs = District_URLs.append({'ID' : idd, 'District': district, 'URL' : url}, ignore_index=True)
    
    time.sleep(random.randint(5,10))

## Step 2: Get all the information of the property

Take Bunkyoku(文京区) for example

In [2]:
Bunkyo_url = District_URLs.loc[4,'URL']
response = rq.get(Bunkyo_url) 
html_doc = response.text 
soup = BeautifulSoup(response.text, "lxml")

### Step 2-1  Get the information on any given page

#### a.部屋の特徴・設備 (Features of the object)

In [4]:
def detail_url(birubody):
    detail = birubody.find("td", class_="ui-text--midium ui-text--bold")
    detail_a = detail.a['href']
    detail_url = 'http://suumo.jp'+ detail_a
    detail_response = rq.get(detail_url)
    detail_soup = BeautifulSoup(detail_response.text, "lxml")
    features = detail_soup.find("div", class_ = "bgc-wht ol-g")
    if (features is None):
        return("No longer availiable")
    else:
        features_text = features.get_text().strip()
        return(features_text)
    time.sleep(random.uniform(3,8))

#### b. Basic information (Rent, Size, Age, etc.)

In [5]:
def getInfo(District_url):
    d = []
    global df
    #s = 0
    response = rq.get(District_url) 
    soup = BeautifulSoup(response.text, "lxml")
    biru_set = soup.find_all("div", "cassetteitem")

    for i in range(len(biru_set)):
        biru_body = biru_set[i].find_all("tbody")
        title = biru_set[i].find("div","cassetteitem_content-title") #Name of the building
        location = biru_set[i].find("li", "cassetteitem_detail-col1") #Location of the building
        district = location.text.split(sep = "区")[0]
        district = district + "区" #Split the location and extract "district" in the address
        age = biru_set[i].find("li", "cassetteitem_detail-col3").find_all("div")[0] #Age of the building
        totalfloor = biru_set[i].find("li", "cassetteitem_detail-col3").find_all("div")[1] #Total floor count of the building
        for j in range(len(biru_body)):
            floor = biru_body[j].find_all("td")[2]
            floortext = floor.text.replace('											', '') #Floor
            rent = biru_body[j].find("span", class_ = "cassetteitem_other-emphasis ui-text--bold") #Rent
            madori = biru_body[j].find("span", class_ = "cassetteitem_madori") #Layout
            menseki = biru_body[j].find("span", class_ = "cassetteitem_menseki") #Area
            detail = detail_url(biru_body[j]) #More detailed information

            d.append((i+1, title.text, district, location.text, floortext.strip(), totalfloor.text, rent.text, madori.text, menseki.text, age.text, detail))

        df = pd.DataFrame(d, columns=['BuildingCount', 'BuildingName', 'District', 'Location', 'Floor', 'TotalFloor','Rent', 'Madori', 'Menseki', 'Age', 'Details'])
    return(df)

### Step 2-2 Turn pages automatically

In [6]:
def TurnPage(District_url):
    global df_everypage 
    df_everypage =  pd.DataFrame()
    response = rq.get(District_url) 
    soup = BeautifulSoup(response.text, "lxml")
    pagesection = soup.find("div", class_ = "pagination pagination_set-nav") # Find the page button section
    totalpagescount = pagesection.find("ol", "pagination-parts").find_all("li")[-1].text # The number on the last page button is the total number of the pages that we have to page through
    
    for p in range(1,int(totalpagescount)+1):#
        urls = Bunkyo_url +'&et=9999999&page1&page='+str(p)
        getInfo(urls)
        df["From Page #"] = p
        df_everypage = df_everypage.append(df)
        time.sleep(random.uniform(5,10))
    return(df_everypage)


## Step 3 Repeat step 2 on every district in Tokyo

In [9]:
def Oneclick(URL_list):
    global Tokyo_Dataset
    Tokyo_Dataset = pd.DataFrame()

    for i in range(0,len(URL_list)):
        D = TurnPage(URL_list.loc[i,'URL'])
        Tokyo_Dataset = Tokyo_Dataset.append(D)
        
    return(Tokyo_DataSet)